<a href="https://colab.research.google.com/github/dude123studios/TradingBots/blob/main/Manifold_Auto_Better.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import yfinance as yf
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from scipy.special import softmax

# Constants
NEWS_API_KEY = "658d2e0e1d4a4203babad16ca1612e72"
MANIFOLD_API_KEY = "a3b83c3a-a0e1-45bb-a998-06b811d5e50f"
FINBERT_MODEL = "ProsusAI/finbert"
STOCKS = ["NVDA", "AAPL", "MSFT", "GOOGL", "TSLA", "AMD", "META", "AMZN", "NFLX", "INTC"]
KEYWORDS = ["NVIDIA", "Apple", "Microsoft", "Google", "Tesla", "AMD", "Meta", "Amazon", "Netflix", "Intel"]

# Load FinBERT
tokenizer = BertTokenizer.from_pretrained(FINBERT_MODEL)
model = BertForSequenceClassification.from_pretrained(FINBERT_MODEL)
model.eval()

# Fetch news for all stocks
def fetch_news():
    query = " OR ".join(KEYWORDS)
    url = f"https://newsapi.org/v2/everything?q={query}&sortBy=popularity&pageSize=50&apiKey={NEWS_API_KEY}"
    response = requests.get(url)
    articles = response.json().get("articles", [])
    return articles

# Filter news per stock
def filter_news_by_stock(articles):
    stock_news = {stock: [] for stock in STOCKS}
    for article in articles:
        title = article["title"] + " " + article.get("description", "")
        for stock, keyword in zip(STOCKS, KEYWORDS):
            if keyword.lower() in title.lower():
                stock_news[stock].append(title)
    return stock_news

# Analyze sentiment using FinBERT
def analyze_sentiment(news_list):
    if not news_list:
        return 0.5
    inputs = tokenizer(news_list, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    scores = softmax(outputs.logits.numpy(), axis=1)
    return scores[:, 2].mean()  # Positive sentiment score

# Get stock prices
def get_stock_prices(stock):
    hist = yf.Ticker(stock).history(period="2d")["Close"].values
    if len(hist) < 2:
        return None, None
    return hist[-2], hist[-1]  # Yesterday's and today's closing prices

# Get Manifold market for a stock
def find_manifold_market(stock):
    query = f"{stock} close higher OR lower"
    url = f"https://api.manifold.markets/v0/markets?search={query}"
    response = requests.get(url).json()
    for market in response:
        if stock in market["question"]:
            return market["id"]
    return None

# Place bet on Manifold
def place_bet(market_id, bet_amount, outcome):
    url = "https://api.manifold.markets/v0/bet"
    headers = {"Authorization": f"Key {MANIFOLD_API_KEY}"}
    data = {"marketId": market_id, "amount": bet_amount, "outcome": outcome}
    response = requests.post(url, headers=headers, json=data)
    return response.json()

# Predict movements and bet
def predict_and_bet():
    articles = fetch_news()
    stock_news = filter_news_by_stock(articles)

    bets = []
    for stock, news in stock_news.items():
        avg_sentiment = analyze_sentiment(news)
        prev_close, today_close = get_stock_prices(stock)

        if prev_close is None:
            continue

        predicted_direction = "UP" if avg_sentiment > 0.5 else "DOWN"
        if avg_sentiment == 0.5:
            continue
        print(f"Bet ${avg_sentiment} on {stock} closing .")
        continue
        market_id = find_manifold_market(stock)
        if not market_id:
            continue

        bet_amount = round(abs(avg_sentiment - 0.5) * 100)  # Confidence-based bet sizing
        outcome = "YES" if predicted_direction == "UP" else "NO"

        result = place_bet(market_id, bet_amount, outcome)
        bets.append((stock, predicted_direction, bet_amount, result))

    return bets

# Run predictions and bets
bets_placed = predict_and_bet()
#for stock, prediction, amount, result in bets_placed:
 #   print(f"Bet ${amount} on {stock} closing {prediction}. Result: {result}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Bet $0.9120645523071289 on GOOGL closing .
